# MNIST Classification

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Importar el csv de entrenamiento a una variable de pandas.
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')


In [ ]:
df_train.shape

In [ ]:
Y_train = df_train['label'].values
X_train = df_train.drop('label', axis=1).values

# X es el conjunto de datos, e Y es la etiqueta
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2)

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
Y_train = torch.from_numpy(Y_train).long()
Y_test = torch.from_numpy(Y_test).long()

X_train = X_train.view(-1, 28, 28)
X_test = X_test.view(-1, 28, 28)


In [ ]:
X_train.shape

In [ ]:
# Validando dataset
img = np.array(X_train[100])
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
Y_train[100]

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
dataloader = {
    'train': torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(Y_train, dtype=torch.long)
        ),
        batch_size=64,
        shuffle=True
    ),
    'test': torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(
            torch.tensor(X_test, dtype=torch.float32),
            torch.tensor(X_test, dtype=torch.long)
        ),
        batch_size=64,
        shuffle=True
    )
}

In [ ]:
# Guardar en una variable un batch del dataloader de entrenamiento.
batch = next(iter(dataloader['train']))
bar = tqdm(dataloader['train'])
# Hacer un reshape del batch para que sea de 64 imagenes de 28x28
batch[0] = batch[0].view(1, 64, 28, 28)
batch[0].shape


In [ ]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
           torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = torch.nn.Linear(14 * 14 * 32, 1000)
        self.fc2 = torch.nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

class CNN_Sensio(torch.nn.Module):
    def __init__(self, n_channels=1, n_outputs=10):
        super().__init__()
        self.conv1 = block(n_channels, 64)
        self.conv2 = block(64, 128)
        self.fc = torch.nn.Linear(128*8*8, n_outputs)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
def fit(model, dataloader, epochs=10):
    
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            ##################################
            optimizer.zero_grad()
            y_hat = model(X.view(1, 64, 28, 28))
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            ##################################
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"## Loss: {np.mean(train_loss):.5f},  ## Acc: {np.mean(train_acc):.5f}")
        
        val_loss, val_acc = [], []
        model.eval()
        bar = tqdm(dataloader['test'])
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.view(64, 1, 28, 28).to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")


In [ ]:
model = CNN_Sensio()
fit(model, dataloader, epochs=20)